In [1]:
from tqdm import tqdm, trange
import numpy as np

import tensorflow as tf
from tensorflow import function, GradientTape, sqrt, abs, reduce_mean, ones_like, zeros_like, convert_to_tensor, float32, config, reshape
from tensorflow import data as tfdata
from tensorflow import nn
from keras import (Model, Sequential, Input)
from keras.layers import (GRU, LSTM, Dense, Flatten)
from tensorflow.keras.optimizers import Adam
from keras.losses import (BinaryCrossentropy, MeanSquaredError)
tf.config.run_functions_eagerly(True)

gpu = tf.config.experimental.list_physical_devices('GPU')
if len(gpu)!=0:
    tf.config.experimental.set_visible_devices(gpu[0], 'GPU')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from support.utils import real_data_loading
import pandas as pd
from sklearn.decomposition import PCA#from sklearn.manifold import TSNE
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt

In [2]:
def get_batch_data(data, batch_size):
        data = convert_to_tensor(data, dtype=float32)
        return iter(tfdata.Dataset.from_tensor_slices(data)
                                .batch(batch_size).repeat(1))
    
def order_batch(data, ind):
        d = []
        for i in data:
            d.append(i[ind])
        return(d)
    
def make_net(model, n_layers, hidden_units, output_units, net_type='GRU'):
    if net_type=='GRU':
        for i in range(n_layers):
            model.add(GRU(units=hidden_units,
                      return_sequences=True,
                      name=f'GRU_{i + 1}'))
    else:
        for i in range(n_layers):
            model.add(LSTM(units=hidden_units,
                      return_sequences=True,
                      name=f'LSTM_{i + 1}'))
    if output_units==1:
        model.add(Flatten())

        model.add(Dense(units=output_units,
                        activation='sigmoid',
                        name='OUT'))
        return(model)

    model.add(Dense(units=output_units,
                    activation='sigmoid',
                    name='OUT'))

    return model

In [3]:
train_path = "data_train_24.csv"
train_df = pd.read_csv(train_path)

test_path = "data_test_24.csv"
test_df = pd.read_csv(test_path)

seq_len = 24

# Data transformations to be applied prior to be used with the synthesizer model
train_data = real_data_loading(train_df.values, seq_len=seq_len, n_signal=3)
test_data = real_data_loading(test_df.values, seq_len=seq_len, n_signal=3)
print(len(train_data))

6619902840
1686491944
3902591


In [4]:
class Recovery(Model):
    def __init__(self, hidden_dim, n_out, n_layers):#, net_type):
        super(Recovery, self).__init__()
        self.hidden_dim=hidden_dim
        self.n_out=n_out
        #self.net_type=net_type
        self.n_layers=n_layers
    
    def call(self,x):
        y = Sequential(name='Recovery')(x)
        
        for i in range(self.n_layers):
            y = GRU(units=self.hidden_dim, return_sequences=True, name=f'GRU_{i + 1}')(y)
            
        y = Flatten()(y)

        return(Dense(units=self.n_out, activation='sigmoid', name='OUT')(y))
        
class Embedder(Model):
    def __init__(self, hidden_dim, n_layers):
        super(Embedder, self).__init__()
        self.hidden_dim=hidden_dim
        self.n_layers=n_layers
    
    def call(self,x):
        y = Sequential(name='Embedder')(x)
        
        for i in range(self.n_layers):
            y = GRU(units=self.hidden_dim, return_sequences=True, name=f'GRU_{i + 1}')(y)

        return(Dense(units=self.hidden_dim, activation='sigmoid', name='OUT')(y))

In [5]:
hidden_dim = n_seq = 8
n_out = 1
batch_size = 16
recovery = Recovery(hidden_dim, n_out, 3)
embedder = Embedder(hidden_dim, 3)

X = Input(shape=[seq_len, n_seq], name='RealData') #, batch_size=batch_size

#--------------------------------
# Building the AutoEncoder
#--------------------------------
H = embedder(X)
X_tilde = recovery(H)

autoencoder = Model(inputs=X, outputs=X_tilde)

In [6]:
loss_object = MeanSquaredError()

optimizer = Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanSquaredError(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanSquaredError(name='test_accuracy')

In [8]:
@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = autoencoder(inputs)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, autoencoder.trainable_variables)
  optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(inputs, labels):
  predictions = autoencoder(inputs)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [10]:
EPOCHS = 2

#for epoch in range(EPOCHS):
for epoch in tqdm(range(EPOCHS), desc='Network training'):
    for images, labels in zip([next(get_batch_data(order_batch(train_data,0), batch_size))], [next(get_batch_data(order_batch(train_data,1), batch_size))]):
        train_step(images, labels)

    for test_images, test_labels in zip([next(get_batch_data(order_batch(test_data,0), batch_size))], [next(get_batch_data(order_batch(test_data,1), batch_size))]):
        test_step(test_images, test_labels)

    template = 'Epoch {}, Perdida: {}, Exactitud: {}, Perdida de prueba: {}, Exactitud de prueba: {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

    # Reinicia las metricas para el siguiente epoch.
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Network training:  50%|████████████████████████████████▌                                | 1/2 [02:43<02:43, 163.59s/it]

Epoch 1, Perdida: 0.19730889797210693, Exactitud: 19.73089027404785, Perdida de prueba: 0.17120860517024994, Exactitud de prueba: 17.120861053466797


Network training: 100%|█████████████████████████████████████████████████████████████████| 2/2 [05:49<00:00, 174.62s/it]

Epoch 2, Perdida: 0.15578711032867432, Exactitud: 15.578710556030273, Perdida de prueba: 0.1359119564294815, Exactitud de prueba: 13.591196060180664
